In [ ]:
class Cargo:
    def __init__(self,name,weight,volume):
        self.name=name
        if weight<0:
            raise ValueError("Введите корретные значения")
        else:
            self.weight=weight
        if volume<0:
            raise ValueError("Введите корретные значения")
        else:
            self.volume=volume
        

class Warehouse:
    def __init__(self,name,location):
        self.name=name
        self.location=location
        self.cargo_list=[]

    def accept_cargo(self,cargo):   
        if type(cargo)==Cargo:
            self.cargo_list.append(cargo)
            return f"Вы добавли {cargo.name} на склад"
        else:
            return "Добавлять можно только обьекты класса Cargo" 
    
    def deliver_cargo(self,cargo):
        flag=False
        for cargo_itter in self.cargo_list:
            if cargo_itter.name==cargo.name:
                self.cargo_list.remove(cargo_itter)
                flag=True
        if flag==True:    
            return f"Вы забрали {cargo.name} со склада"
        else:
            return"Груза нет на складе"
    
    def cargo_availability(self,cargo):
        flag=False
        for cargo_itter in self.cargo_list:
            if cargo_itter.name==cargo.name:
                flag=True
        return flag

class Transport:
    def __init__(self,name,max_weight,max_volume,average_speed_kmh,price_per_hour):
        self.name=name
        self.max_weight=max_weight
        self.max_volume=max_volume
        self.average_speed_kmh=average_speed_kmh
        self.price_per_hour=price_per_hour

    def calculate_time(self,warehouse1,warehouse2):
        distance=((warehouse1.location[0]-warehouse2.location[0])**2+(warehouse1.location[1]-warehouse2.location[1])**2)**0.5
        time=distance/self.average_speed_kmh
        return time
    
    def calculate_cost(self,warehouse1,warehouse2):
        distance=((warehouse1.location[0]-warehouse2.location[0])**2+(warehouse1.location[1]-warehouse2.location[1])**2)**0.5
        time=distance/self.average_speed_kmh
        cost_of_transport=time*self.price_per_hour
        return cost_of_transport
    
    def can_transport(self,cargo):
        flag=False
        if self.max_weight >= cargo.weight and self.max_volume >= cargo.volume:
            flag=True 
        return flag
    
class Truck(Transport):
    def __init__(self,name,max_weight,max_volume,average_speed_kmh,price_per_hour):
        super().__init__(name,max_weight,max_volume,average_speed_kmh,price_per_hour)

class Aircraft(Transport):
    def __init__(self,name,max_weight,max_volume,average_speed_kmh,price_per_hour):
        super().__init__(name,max_weight,max_volume,average_speed_kmh,price_per_hour)

class Ship(Transport):
    def __init__(self,name,max_weight,max_volume,average_speed_kmh,price_per_hour):
        super().__init__(name,max_weight,max_volume,average_speed_kmh,price_per_hour)

class Route:
    def __init__(self, transport, warehouse_from, warehouse_to, cargos):
        self.transport = transport
        self.warehouse_from = warehouse_from
        self.warehouse_to = warehouse_to
        self.cargos = list(cargos)  

    def add_cargo(self, cargo):
        if type(cargo) == Cargo:
            self.cargos.append(cargo)
            print(f"Вы добавили {cargo.name} в маршрут")
        else:
            raise TypeError("Можно добавлять только объекты Cargo")    
    
    def remove_cargo(self, cargo):
        for cargo_item in self.cargos:
            if cargo_item.name == cargo.name:
                self.cargos.remove(cargo_item)
                return f"Вы забрали {cargo.name} из транспорта"
        return "Груза нет в маршруте"
        
    def calculate_time(self):
        return self.transport.calculate_time(self.warehouse_from, self.warehouse_to)

    def calculate_cost(self):
        return self.transport.calculate_cost(self.warehouse_from, self.warehouse_to)

    def validate(self):
        total_weight = sum(cargo_item.weight for cargo_item in self.cargos)
        total_volume = sum(cargo_item.volume for cargo_item in self.cargos)

        if total_weight > self.transport.max_weight:
            print(f"Превышена грузоподъёмность: {total_weight} > {self.transport.max_weight}")
            return False
        if total_volume > self.transport.max_volume:
            print(f"Превышен объём: {total_volume} > {self.transport.max_volume}")
            return False

        return True

    def is_execute(self):
        if not self.validate():
            return False
        for cargo in self.cargos:
            if not self.warehouse_from.cargo_availability(cargo):
                return False
        return True

    def execute(self):
        if not self.is_execute():  
            return {"success": False, "message": "Невозможно выполнить маршрут: проверьте грузы и транспорт"}

        for cargo in self.cargos[:]:
            result = self.warehouse_from.deliver_cargo(cargo)
            if "нет на складе" in result:
                return {"success": False, "message": f"Не удалось забрать груз '{cargo.name}'"}

        for cargo in self.cargos:
            result = self.warehouse_to.accept_cargo(cargo)
            if "только объекты класса Cargo" in result:
                return {"success": False, "message": f"Ошибка при добавлении груза '{cargo.name}'"}

        return {
            "success": True,
            "message": f"Маршрут выполнен: {len(self.cargos)} грузов доставлено",
            "time_hours": self.calculate_time(),
            "cost": self.calculate_cost()
        }
 
class LogisticsManager():
    def __init__(self,transports=[]):
        self.transports=transports

    def add_transport(self, transport):
        if isinstance(transport, Transport):
            self.transports.append(transport)
        else:
            raise TypeError("Можно добавлять только объекты типа Transport")
        
    def find_suitable_transport(self,cargos):
        total_weight = sum(cargo.weight for cargo in cargos)
        total_volume = sum(cargo.volume for cargo in cargos)

        for transport in self.transports:
            if transport.max_weight >= total_weight and transport.max_volume >= total_volume:
                return transport
        print("Нет подходящего транспорта")
        return None

    def create_delivery(self,warehouse_from,warehouse_to,cargos):
        transport=self.find_suitable_transport(cargos)
        if not transport:
            return {
                "success": False,
                "message": "Нет подходящего транспорта"
            }


        route=Route(transport,warehouse_from,warehouse_to,cargos)

        if not route.is_execute():
            missing_cargos = []
            for cargo in cargos:
                if not warehouse_from.cargo_availability(cargo):
                    missing_cargos.append(cargo.name)
            if missing_cargos:
                return {
                    "success": False,
                    "message": f"Грузы отсутствуют на складе: {', '.join(missing_cargos)}"
                }
            else:
                return {
                    "success": False,
                    "message": "Маршрут не может быть выполнен (проверьте вес/объём грузов)"
                }
        

        result=route.execute()
        return result
        
class MultiRoute:
    def __init__(self, routes):
        if type(routes) is not list:
            raise ValueError("routes должен быть списком объектов Route")
        if len(routes) == 0:
            raise ValueError("routes не должен быть пустым")
        self.routes = routes
        self.status = "планируется"

    def validate_chain(self):
        first_route = self.routes[0]
        for cargo in first_route.cargos:
            if not first_route.warehouse_from.cargo_availability(cargo):
                print(f"Ошибка: груз '{cargo.name}' отсутствует на начальном складе '{first_route.warehouse_from.name}'")
                return False

        for i in range(1, len(self.routes)):
            prev_route = self.routes[i - 1]
            curr_route = self.routes[i]
            if prev_route.warehouse_to is not curr_route.warehouse_from:
                print(f"Ошибка: нарушена связность этапов!")
                print(f"  Этап {i+1} должен начинаться с '{prev_route.warehouse_to.name}', но начинается с '{curr_route.warehouse_from.name}'")
                return False

        for i, route in enumerate(self.routes):
            if not route.validate():
                print(f"Ошибка: этап {i+1} не прошёл валидацию")
                return False

        return True

    def calculate_total_time(self):
        return sum(route.calculate_time() for route in self.routes)

    def calculate_total_cost(self):
        return sum(route.calculate_cost() for route in self.routes)
    
    def execute(self):
        self.status = "в процессе"

        if not self.validate_chain():
            self.status = "провалено"
            return {"success": False, "message": "Ошибка валидации цепочки (см. консоль)"}

        for i, route in enumerate(self.routes):
            result = route.execute()
            if not result["success"]:
                self.status = "провалено"
                return {
                    "success": False,
                    "message": f"Этап {i+1} не выполнен: {result['message']}",
                    "failed_at_stage": i + 1
                }

        self.status = "завершено"
        total_time = self.calculate_total_time()
        total_cost = self.calculate_total_cost()

        return {
            "success": True,
            "message": f"Мультидоставка успешно завершена: {len(self.routes)} этапов",
            "total_time_hours": round(total_time, 2),
            "total_cost": round(total_cost, 2)
        }


Пример работы

In [ ]:

factory = Warehouse("Завод Электроники", (0, 0))
distribution_center = Warehouse("Распределительный центр", (100, 0))
regional_store = Warehouse("Региональный магазин", (100, 150))

laptops = Cargo("Ноутбуки", 500, 10)
monitors = Cargo("Мониторы", 300, 8)
accessories = Cargo("Аксессуары", 100, 3)

print(factory.accept_cargo(laptops))
print(factory.accept_cargo(monitors))
print(factory.accept_cargo(accessories))

truck = Truck("Грузовик DAF", 1000, 25, 60, 50)
aircraft = Aircraft("Самолёт Boeing", 5000, 100, 800, 500)
ship = Ship("Контейнеровоз", 10000, 500, 30, 20)

logistics = LogisticsManager()
logistics.add_transport(truck)
logistics.add_transport(aircraft)
logistics.add_transport(ship)


print( " Доставка всех грузов с завода на распределительный центр")
cargos_stage1 = [laptops, monitors, accessories]
result1 = logistics.create_delivery(factory, distribution_center, cargos_stage1)
print(result1)


print(" Доставка ноутбуков и аксессуаров в магазин")
cargos_stage2 = [laptops, accessories]
result2 = logistics.create_delivery(distribution_center, regional_store, cargos_stage2)
print(result2)

print(" Мультидоставка мониторов из распределительного центра в магазин")
monitors_for_multi = Cargo("Мониторы", 300, 8)
if not distribution_center.cargo_availability(monitors_for_multi):
    distribution_center.accept_cargo(monitors_for_multi)

route_multi = Route(aircraft, distribution_center, regional_store, [monitors_for_multi])

multi_route = MultiRoute([route_multi])

result_multi = multi_route.execute()
print(result_multi)


print(f" Завод Электроники: {[c.name for c in factory.cargo_list]}")
print(f"Распределительный центр: {[c.name for c in distribution_center.cargo_list]}")
print(f"Региональный магазин: {[c.name for c in regional_store.cargo_list]}")


Тест ошибок

In [ ]:
bad_cargo = Cargo("Невозможный груз", weight=-100, volume=5)

In [ ]:
not_cargo = "Я просто строка"
result = warehouse_A.accept_cargo(not_cargo)
print(result)

In [ ]:
ghost_cargo = Cargo("Привидение", weight=1, volume=1)
result = warehouse_A.deliver_cargo(ghost_cargo)
print(result)

In [ ]:
route = Route(truck, warehouse_A, warehouse_B, [cargo1])
route.add_cargo("не груз")

In [ ]:
result = route.remove_cargo(ghost_cargo)
print(result)

In [ ]:
overweight_route = Route(truck, warehouse_A, warehouse_B, [cargo1, cargo2])
result = overweight_route.execute()
print(result["message"])

In [ ]:
oversize_route = Route(truck, warehouse_A, warehouse_B, [cargo2])

result = oversize_route.execute()
print(result["message"])

In [ ]:
missing_route = Route(truck, warehouse_A, warehouse_B, [cargo3])

result = missing_route.execute()
print(result["message"])

In [ ]:
too_heavy_cargo = Cargo("Кран", weight=20000, volume=100)
result = logistics_manager.create_delivery(warehouse_A, warehouse_B, [too_heavy_cargo])
print(result["message"])

In [ ]:
bad_multiroute = MultiRoute([])

In [ ]:
route_A_B = Route(truck, warehouse_A, warehouse_B, [cargo1])
route_C_B = Route(truck, warehouse_C, warehouse_B, [cargo1])

broken_chain = MultiRoute([route_A_B, route_C_B])
result = broken_chain.execute()
print(result["message"])

In [ ]:
warehouse_A.accept_cargo(cargo4)

route1 = Route(truck, warehouse_A, warehouse_B, [cargo4])
route2 = Route(truck, warehouse_B, warehouse_C, [cargo4])

route1.execute() 

warehouse_B.deliver_cargo(cargo4)

sabotaged_multiroute = MultiRoute([route1, route2])
result = sabotaged_multiroute.execute()
print(result["message"])
print("упал на этапе:", result["failed_at_stage"])

In [ ]:
not_transport = "Я не транспорт"
logistics_manager.add_transport(not_transport)